In [1]:
from happytransformer import HappyTextToText
from datasets import load_dataset
import csv
import torch
import os
#os.environ['CUDA_VISIBLE_DEVICES'] ='0'

2022-12-01 00:20:55.669026: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 00:20:55.851133: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-01 00:20:57.542087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/software/software/ZeroMQ/4.3.4-GCCcore-11.2.0/lib:/opt/software/software/util-linu

In [2]:
happy_tt = HappyTextToText("T5", "t5-base")

12/01/2022 00:21:22 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [ ]:
from datasets import load_dataset
dataset = load_dataset("jfleg")

In [3]:
train_dataset = load_dataset("jfleg", split='validation[:]')
eval_dataset = load_dataset("jfleg", split='test[248:]')
train_dataset2 = load_dataset("jfleg", split='test[:248]')

In [4]:
print(train_dataset)

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 755
})


In [5]:
print(eval_dataset)

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 500
})


In [6]:
print(train_dataset2)

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 248
})


In [7]:
for case in train_dataset["corrections"][:2]:
  print(case)
  print(case[0])

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 


In [8]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = case["sentence"]
            for correction in case["corrections"]:
                if input_text and correction:
                    input_text = remove_excess_spaces(input_text)
                    correction = remove_excess_spaces(correction)
                    writter.writerow([input_text, correction])
              

In [9]:
generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)
generate_csv("train2.csv", train_dataset2)
import pandas as pd
df = pd.concat(map(pd.read_csv, ['train.csv', 'train2.csv']), ignore_index=True)
df.to_csv('new_train.csv',index=False)

In [10]:
before_result = happy_tt.eval("eval.csv")

12/01/2022 00:21:35 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/p793x363/.cache/huggingface/datasets/csv/default-d98a1db4dd444943/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/p793x363/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
***** Running Evaluation *****
  Num examples = 1996
  Batch size = 1


In [11]:
print("Before loss: ", before_result.loss)

Before loss:  1.5075130462646484


In [12]:
from happytransformer import TTTrainArgs

In [13]:
args = TTTrainArgs(batch_size=8,num_train_epochs=30)

In [14]:
happy_tt.train("new_train.csv", args=args)

12/01/2022 00:25:42 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/p793x363/.cache/huggingface/datasets/csv/default-3dbc681b41eff753/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

12/01/2022 00:25:46 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
***** Running training *****
  Num examples = 4008
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5010


Step,Training Loss
500,0.639300
1000,0.470300
1500,0.402700
2000,0.361400
2500,0.331100
3000,0.307900
3500,0.292800
4000,0.277500
4500,0.264800
5000,0.258600




Training completed. Do not forget to share your model on huggingface.co/models =)




In [15]:
after_result = happy_tt.eval("eval.csv")

12/01/2022 02:11:42 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 1996
  Batch size = 1


In [16]:
print("After loss:", after_result.loss)

After loss: 0.490283727645874


In [23]:
happy_tt.save("Model_gec/")

Configuration saved in Model_gec2/config.json
Model weights saved in Model_gec2/pytorch_model.bin
tokenizer config file saved in Model_gec2/tokenizer_config.json
Special tokens file saved in Model_gec2/special_tokens_map.json
Copy vocab file to Model_gec2/spiece.model


In [2]:
from happytransformer import HappyTextToText
from happytransformer import TTSettings

happy_tt = HappyTextToText("T5", "t5-base",load_path='Model_gec')
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)
text = 'He are moving here.'
result = happy_tt.generate_text(text, args=beam_settings)
print(result.text)

12/01/2022 08:35:48 - INFO - happytransformer.happy_transformer -   Using model: cpu


He is moving here.
